In [ ]:
# Imports

import torch
import matplotlib.pyplot as plt
from datetime import datetime

%run prep_clean.ipynb
%run prep_noise.ipynb

In [4]:
def gen_data(heterogeneous=False, nsr=0.0, seed=None, visualize=False):
    n_clients = 8
    (Xtrain, ytrain_clean), (Xval, yval_clean), (Xtest, ytest_clean) = get_clean_data(heterogeneous)
    load_schedules, noise_var, (ytrain_noise, yval_noise) = get_load_dynamics(nsr, seed, visualize)
    ytrain_noisy = ytrain_clean + ytrain_noise
    yval_noisy = yval_clean + yval_noise
    
    load_schedules_ls = load_schedules.tolist()
    noise_var_ls = noise_var.tolist()
    Xtrain_ls = Xtrain.tolist()
    Xval_ls = Xval.tolist()
    Xtest_ls = Xtest.tolist()
    ytrain_ls = ytrain_noisy.tolist()
    yval_ls = yval_noisy.tolist()
    ytest_ls = ytest_clean.tolist()
        
    bs_sim = {}

    bs_sim["load_schedules"] = dict(zip(range(len(load_schedules_ls)), load_schedules_ls))
    bs_sim["noise_var_schedules"] = dict(zip(range(len(noise_var_ls)), noise_var_ls))
    
    bs_sim["data"] = {i: {"train": {"x": Xtrain_ls[i], "y": ytrain_ls[i]},
                        "val": {"x": Xval_ls[i], "y": yval_ls[i]},
                         "test": {"x": Xtest_ls[i], "y": ytest_ls[i]}} for i in range(n_clients)}
    
    out_path = "/proj/fair-ai/fair-fl/simulation"
    
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    
    dist_mode = "het" if heterogeneous else "hom"
    file_path = os.path.join(out_path, f"bike_{dist_mode}.json")
    
    with open(file_path,'w') as outfile:
        json.dump(bs_sim, outfile)
        
    print("Saved to:", file_path, "at", datetime.now())
    print("NSR:", nsr)
    print("Seed:", seed)
    
    return (
        load_schedules,
        noise_var,
        (Xtrain, ytrain_noisy, ytrain_clean, ytrain_noise),
        (Xval, yval_noisy, yval_clean, yval_noise),
        (Xtest, ytest_clean)
    )


def generate_experiment_data():
    # Heterogeneous vs Homogeneous
    distribution_modes = [True, False] # True for heterogeneous, False for homogeneous
    distribution_labels = ["bike_het", "bike_hom"]

    # Different NSR levels
    nsr_levels = [0.1, 0.2, 0.5, 1.0]

    # Seeds for data generation
    seeds = list(range(11))

    # Root directory for all data
    root_dir = "/proj/fair-ai/fair-fl/simulation"

    # Iterate through distribution modes and nsr_levels
    for dist_mode, dist_label in zip(distribution_modes, distribution_labels):
        for nsr in nsr_levels:
            # Create the folder structure: [heterogeneous vs homogeneous]/[nsr_level]/
            nsr_folder_path = os.path.join(root_dir, dist_label, f"nsr_{nsr}")
            if not os.path.exists(nsr_folder_path):
                os.makedirs(nsr_folder_path)

            # Generate data for each seed
            for seed in seeds:
                gen_data(heterogeneous=dist_mode, nsr=nsr, seed=seed, visualize=False)
                # Rename the generated file to include the seed in its name
                original_file_path = os.path.join(root_dir, f"bike_{'het' if dist_mode else 'hom'}.json")
                new_file_path = os.path.join(nsr_folder_path, f"seed_{seed}.json")
                os.rename(original_file_path, new_file_path)


In [5]:
generate_experiment_data()

Saved to: /proj/fair-ai/fair-fl/simulation/bike_het.json at 2023-08-19 11:04:14.640399
NSR: 0.1
Seed: 0
Saved to: /proj/fair-ai/fair-fl/simulation/bike_het.json at 2023-08-19 11:04:31.205538
NSR: 0.1
Seed: 1
Saved to: /proj/fair-ai/fair-fl/simulation/bike_het.json at 2023-08-19 11:04:47.834427
NSR: 0.1
Seed: 2
Saved to: /proj/fair-ai/fair-fl/simulation/bike_het.json at 2023-08-19 11:05:04.412644
NSR: 0.1
Seed: 3
Saved to: /proj/fair-ai/fair-fl/simulation/bike_het.json at 2023-08-19 11:05:21.142654
NSR: 0.1
Seed: 4
Saved to: /proj/fair-ai/fair-fl/simulation/bike_het.json at 2023-08-19 11:05:37.677419
NSR: 0.1
Seed: 5
Saved to: /proj/fair-ai/fair-fl/simulation/bike_het.json at 2023-08-19 11:05:54.300460
NSR: 0.1
Seed: 6
Saved to: /proj/fair-ai/fair-fl/simulation/bike_het.json at 2023-08-19 11:06:10.900678
NSR: 0.1
Seed: 7
Saved to: /proj/fair-ai/fair-fl/simulation/bike_het.json at 2023-08-19 11:06:27.372787
NSR: 0.1
Seed: 8
Saved to: /proj/fair-ai/fair-fl/simulation/bike_het.json at 2023